<a href="https://colab.research.google.com/github/PavelStafeev/postomats/blob/main/model_prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Описание прототипа модели https://docs.google.com/presentation/d/1afds7A37Q5-upmaPRe_-b6L4grZfu4-0c1eXUpw4n4U/edit?usp=sharing 

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

In [5]:
project_dir = ('/content/drive/My Drive/Colab Notebooks/points')

In [ ]:
import json 

In [ ]:
!pip install geopandas

In [ ]:
import geopandas as gpd

In [ ]:
import gzip
with gzip.open(project_dir + '/sssl.geojson.gz', 'rb') as f:
    file_content = gpd.read_file(f)

In [ ]:
file_content

In [ ]:
!pip install geoplot

In [ ]:
import geoplot
import geoplot.crs as gcrs

In [ ]:
import folium

Выделяем границы окргов города Москвы и кладем их в переменную m

In [ ]:
def visualize_polygons(geometry):
    
    lats, lons = get_lat_lon(geometry)
    
    m = folium.Map(location=[sum(lats)/len(lats), sum(lons)/len(lons)], zoom_start=13, tiles='cartodbpositron')
    
    overlay = gpd.GeoSeries(geometry).to_json()
    folium.GeoJson(overlay, name = 'boundary').add_to(m)
    
    return m

# выводим центроиды полигонов
def get_lat_lon(geometry):
        
    lon = geometry.apply(lambda x: x.x if x.type == 'Point' else x.centroid.x)
    lat = geometry.apply(lambda x: x.y if x.type == 'Point' else x.centroid.y)
    return lat, lon
  

visualize_polygons(file_content['geometry'])

In [ ]:
import numpy as np

In [ ]:
!pip install h3
import h3

In [ ]:
!pip install osmnx
import json
import osmnx as ox
from shapely import wkt
from folium.plugins import HeatMap
from shapely.geometry import Polygon

Выделяем границы окргов города Москвы и кладем их в переменную m

In [ ]:
def visualize_polygons(geometry):
    
    lats, lons = get_lat_lon(geometry)
    
    m = folium.Map(location=[sum(lats)/len(lats), sum(lons)/len(lons)], zoom_start=13, tiles='cartodbpositron')
    
    overlay = gpd.GeoSeries(geometry).to_json()
    folium.GeoJson(overlay, name = 'boundary').add_to(m)
    
    return m

# выводим центроиды полигонов
def get_lat_lon(geometry):
        
    lon = geometry.apply(lambda x: x.x if x.type == 'Point' else x.centroid.x)
    lat = geometry.apply(lambda x: x.y if x.type == 'Point' else x.centroid.y)
    return lat, lon
  

m = visualize_polygons(file_content['geometry'])

Строим гексагоны в границах полигонов 

In [ ]:
def create_hexagons(geoJson): # в качестве параметра получаем geoJson
    

    hexagons = list(h3.polyfill(geoJson, 8)) # Получить все шестиугольники с центрами содержащимися в заданном многоугольнике, словарь переделывем в список
    polylines = [] # создаем пустой список polylines
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False) #создает мультиполигон, описывающий контурыф набора шестиугольников
        # flatten polygons into loops. 
        outlines = [loop for polygon in polygons for loop in polygon] #трансформируем концтуры набора шестиугольников и изменяемый список???
        polyline = [outline + [outline[0]] for outline in outlines][0] # начальную точку контуров набора шистиугольников добаляем как конечную
        polylines.append(polyline) # добавляем конутуры набора шестиугольников в переменную polylines
        
    polylines_x = []
    for j in range(len(polylines)):
        a = np.column_stack((np.array(polylines[j])[:,1],np.array(polylines[j])[:,0])).tolist()
        polylines_x.append([(a[i][0], a[i][1]) for i in range(len(a))])
        
    polygons_hex = pd.Series(polylines_x).apply(lambda x: Polygon(x))
        
    return polylines, polygons_hex
# polygon_hex , polylines - геометрии гексагонов в разных форматах

# сгенерим гексагоны внутри полигона г. Краснодар
geoJson = json.loads(gpd.GeoSeries(file_content['geometry']).to_json()) # Выбираем geometry и трансоформируем в geoseries, трансофрмируем в json, json трансформируем в словарь питона
geoJson = geoJson['features'][6]['geometry'] #из словаря выбираем features c индексом 0, из списка features с индексо 0 выбираем geometry
geoJson = {'type':'Polygon','coordinates': [np.column_stack((np.array(geoJson['coordinates'][0])[:, 1],
                                                      np.array(geoJson['coordinates'][0])[:, 0])).tolist()]} # меняем местами координаты
          
lines, gons = create_hexagons(geoJson)
#sgons = gpd.GeoSeries(gons)
sgons = sgons.append(gpd.GeoSeries(gons)) 
#slines = lines
for n in lines:
  slines.append(n)





Строим гексагоны в границах мультиполигонов 

In [ ]:
def create_hexagons(geoJson): # в качестве параметра получаем geoJson
    

    hexagons = list(h3.polyfill(geoJson, 8)) # Получить все шестиугольники с центрами содержащимися в заданном многоугольнике, словарь переделывем в список
    polylines = [] # создаем пустой список polylines
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False) #создает мультиполигон, описывающий контурыф набора шестиугольников
        # flatten polygons into loops. 
        outlines = [loop for polygon in polygons for loop in polygon] #трансформируем концтуры набора шестиугольников и изменяемый список???
        polyline = [outline + [outline[0]] for outline in outlines][0] # начальную точку контуров набора шистиугольников добаляем как конечную
        polylines.append(polyline) # добавляем конутуры набора шестиугольников в переменную polylines
        
    polylines_x = []
    for j in range(len(polylines)):
        a = np.column_stack((np.array(polylines[j])[:,1],np.array(polylines[j])[:,0])).tolist()
        polylines_x.append([(a[i][0], a[i][1]) for i in range(len(a))])
        
    polygons_hex = pd.Series(polylines_x).apply(lambda x: Polygon(x))
        
    return polylines, polygons_hex
# polygon_hex , polylines - геометрии гексагонов в разных форматах

# сгенерим гексагоны внутри полигона г. Краснодар
geoJson = json.loads(gpd.GeoSeries(file_content['geometry']).to_json()) # Выбираем geometry и трансоформируем в geoseries, трансофрмируем в json, json трансформируем в словарь питона
geoJson = geoJson['features'][8]['geometry'] #из словаря выбираем features c индексом 0, из списка features с индексо 0 выбираем geometry
geoJson = geoJson['coordinates'][1]
geoJson = {'type':'Polygon','coordinates': [np.column_stack((np.array(geoJson[0])[:, 1],
                                                      np.array(geoJson[0])[:, 0])).tolist()]} # меняем местами координаты

lines, gons = create_hexagons(geoJson)
#sgons = gpd.GeoSeries(gons)
sgons = sgons.append(gpd.GeoSeries(gons)) 
#slines = lines
for n in lines:
  slines.append(n)

In [ ]:
with open(project_dir+"/slines_with_comma.txt", "w") as outfile:
  outfile.write(",".join(str(item) for item in slines))

In [ ]:
sgons.reset_index(drop=True, inplace=True)

In [ ]:
sgons.tail()

1543    POLYGON ((37.54863 55.79833, 37.55305 55.79441...
1544    POLYGON ((37.55982 55.84085, 37.56774 55.84132...
1545    POLYGON ((37.55119 55.81104, 37.54677 55.81496...
1546    POLYGON ((37.49930 55.81212, 37.50278 55.81651...
1547    POLYGON ((37.51324 55.82969, 37.52116 55.83017...
dtype: geometry

In [ ]:
sgons.to_file(project_dir+'/polygons.geojson', driver='GeoJSON')  


Добвляем на карты полученные гексагоны

In [ ]:
for s in slines:
  my_PolyLine=folium.PolyLine(locations=s,weight=3,color='red') #задем границы 
  m.add_child(my_PolyLine)

In [ ]:
m

In [ ]:
data_mos = pd.read_csv(project_dir+'/data_mos.csv')

In [112]:
data_mos

,Id,Title
0,1046,Ответственные балансодержатели межрельсового п...
1,629,Спортивные объекты города Москвы
2,673,Точки продажи билетов для проезда в общественн...
3,588,(Архив) Объекты гаражного назначения ГУП «Дире...
4,589,Органы записи актов гражданского состояния (ст...
...,...,...
553,62201,Железнодорожные вокзалы Москвы
554,62603,Карта улиц с односторонним движением в городе ...
555,62904,Ночные маршруты наземного городского пассажирс...
556,61881,(Архив) График закрытия акушерских стационаров...


Выберем объеты систематичного(ежедневного) спроса

In [145]:
data_moss_111 = data_mos.iloc[[1,21,47,192,213,223,224,227,228,231,236,237,241,244,248,250,252,260,269,288,297,311,355,364,378,421,469,473,480,499]]

In [146]:
data_moss_111

,Id,Title
1,629,Спортивные объекты города Москвы
21,556,(Архив) Образовательные учреждения города Моск...
47,540,(Архив) Детские сады
192,1795,(Архив) Комбинаты питания
213,1903,Общественное питание в Москве
223,537,(Архив) Учреждения высшего профессионального о...
224,544,(Архив) Кадетские школы
227,546,(Архив) Колледжи
228,552,(Архив) Лицей
231,1792,(Архив) Кафе


In [ ]:
from zipfile import ZipFile as zf

In [ ]:
json_file = project_dir + "/data-mos-ru-0-100000.zip"

In [ ]:
json_file = zf(json_file)

In [158]:
for text_file in json_file.infolist():
  if text_file.filename.endswith('2263.json'):
    json2263=gpd.read_file(json_file.open(text_file.filename))
    break
  else:
    continue

In [159]:
json2263

,ShortName,OrgType,FullName,WebSite,Subordination,OGRN,INN,KPP,LegalOrganization,LegalAddress,ChiefName,EducationPrograms,global_id,geometry
0,ГБОУ ДО ДТДМ «Хорошево»,организация дополнительного образования,Государственное бюджетное образовательное учре...,dtim.mskobr.ru,Городское подчинение ДОНМ,1037739525273,7734077088,773401001,бюджетное,"123154, ГОРОД МОСКВА, УЛИЦА МАРШАЛА ТУХАЧЕВСКО...",Ледовская Татьяна Леонидовна,Дополнительное образование | дополнительное об...,658727748,"POLYGON ((37.47940 55.78360, 37.47888 55.78369..."
1,ГБУ «Лаборатория путешествий»,прочее,Государственное бюджетное учреждение города Мо...,lab-putesh.mskobr.ru,Городское подчинение ДОНМ,1097746031393,7725663400,770901001,бюджетное,"109052, ГОРОД МОСКВА, УЛИЦА НИЖЕГОРОДСКАЯ, ДОМ 3",Шпаро Матвей Дмитриевич,Профессиональное обучение # Дополнительное обр...,658727903,"POLYGON ((37.67580 55.73956, 37.67559 55.73959..."
2,ГБПОУ колледж «Царицыно»,профессиональная образовательная организация,Государственное бюджетное профессиональное обр...,collegetsaritsyno.mskobr.ru,Городское подчинение ДОНМ,1037739222070,7737057354,772401001,бюджетное,"115569, ГОРОД МОСКВА, ПРОЕЗД ШИПИЛОВСКИЙ, 37, 1",Седова Наталья Николаевна,Профессиональное образование | среднее професс...,658727904,"MULTIPOLYGON (((37.71597 55.62112, 37.71596 55..."
3,ГБОУДО «ДТДиМ «Преображенский»,организация дополнительного образования,Государственное бюджетное образовательное учре...,dtdimvouo.mskobr.ru,Городское подчинение ДОНМ,1037739236590,7718191202,771801001,бюджетное,"107553, ГОРОД МОСКВА, УЛИЦА ЧЕРКИЗОВСКАЯ Б., 15",Коминова Елена Борисовна,Дополнительное образование | дополнительное об...,658727905,"POLYGON ((37.73285 55.80243, 37.73273 55.80245..."
4,ГБПОУ ФК № 35,профессиональная образовательная организация,Государственное бюджетное профессиональное обр...,35finance.mskobr.ru,Городское подчинение ДОНМ,1037739651267,7737014368,772401001,бюджетное,"115404, ГОРОД МОСКВА, УЛИЦА ЛИПЕЦКАЯ, 2, 13",Иванова Елена Анатольевна,Профессиональное обучение # Профессиональное о...,658727972,"POLYGON ((37.66745 55.60456, 37.66735 55.60467..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,ГБОУ школа № 97,общеобразовательная организация,Государственное бюджетное общеобразовательное ...,sch97z.mskobr.ru,Городское подчинение ДОНМ,1127747084519,7731435497,773101001,бюджетное,"121357, ГОРОД МОСКВА, УЛИЦА КРЕМЕНЧУГСКАЯ, 46",Мальчевская Ирина Витальевна,Общее образование | дошкольное образование # О...,2387128079,"MULTIPOLYGON (((37.46449 55.72495, 37.46447 55..."
581,ГБОУ школа № 1252 имени Сервантеса,общеобразовательная организация,Государственное бюджетное общеобразовательное ...,sch1252.mskobr.ru,Городское подчинение ДОНМ,1027739921274,7743022631,774301001,бюджетное,"125080, ГОРОД МОСКВА, УЛИЦА ДУБОСЕКОВСКАЯ, ДОМ 3",Анурова Ирина Владимировна,Общее образование | дошкольное образование # О...,2389604370,"MULTIPOLYGON (((37.49293 55.81340, 37.49302 55..."
582,ГБОУ школа № 2007 ФМШ,общеобразовательная организация,Государственное бюджетное общеобразовательное ...,schuuz2007.mskobr.ru,Городское подчинение ДОНМ,1047796157936,7727506272,772701001,бюджетное,"117042, ГОРОД МОСКВА, УЛИЦА ГОРЧАКОВА, 9, 1",Старовойт Сергей Георгиевич,Общее образование | основное общее образование...,2389604371,"MULTIPOLYGON (((37.54437 55.54312, 37.54442 55..."
583,ГБОУ школа № 1253,общеобразовательная организация,Государственное бюджетное общеобразовательное ...,sch1253c.mskobr.ru,Городское подчинение ДОНМ,1037700049012,7704178709,770401001,бюджетное,"119021, ГОРОД МОСКВА, БУЛЬВАР ЗУБОВСКИЙ, ДОМ 5...",Акулова Наталья Андреевна,Общее образование | дошкольное образование # О...,2389604372,"MULTIPOLYGON (((37.59201 55.73512, 37.59249 55..."


Выводим информацию по многоэтажным домам

In [6]:
buildings = pd.read_excel(project_dir + "/living_buildings.xlsx")

In [8]:
buildings.head(5)

,№ - ТОП,№,Адрес,Площадь м2,Год,Этажей,Подьездов,Помещений / Квартир
0,1.0,18803.0,"пр-кт. Мира, д. 188 Б, к. 1, Москва",54804.10,2014,58,1,370
1,2.0,18805.0,"пр-кт. Мира, д. 188Б, к. 3, Москва",53253.20,2015,58,1,356
2,3.0,29988.0,"пер. Чапаевский, д. 3, Москва",256740.60,2006,57,14,992
3,4.0,19390.0,"ул. Мосфильмовская, д. 8, Москва",137500.80,2011,53,2,564
4,5.0,19431.0,"ул. Мосфильмовская, д. 70, Москва",341272.69,2006,50,7,1046


Дальнейшие шаги к завершению реализации модели указаны в презентации